# Import Modules

In [41]:
import json

import numpy as np
import pandas as pd
import torch

import utils

# Set Vegetable

In [42]:
target_vegetable = utils.VEGETABLES[14]

In [43]:
json_open = open('./free_params.json', 'r')
params = json.load(json_open)
param = params[target_vegetable]

In [44]:
print(f"vegetable: {target_vegetable}")
print(f"free params: {param}")

vegetable: そらまめ
free params: {'T': 10, 'batch_size': 1054, 'num_epochs': 100, 'learning_rate': 0.0005, 'weight_decay': 0.001, 'eps': 1e-08, 'hidden_size': 500, 'dropout_ratio': 0.1}


# Load Data

In [45]:
train_test = pd.read_csv("./data/mapped_train_test.csv")
train_test["date"] = pd.to_datetime(train_test["date"], format="%Y-%m-%d")
weather = pd.read_csv("./data/sorted_mapped_adjusted_weather.csv")
train_test = pd.concat([train_test, weather], axis=1)
train_test = train_test.reset_index(drop=True)

train_test["year"] = train_test.date.dt.year
years = pd.get_dummies(train_test["year"])
train_test = train_test.drop(columns="year")
train_test = pd.concat([train_test, years], axis=1)

train_test["month"] = train_test.date.dt.month
months = pd.get_dummies(train_test["month"])
train_test = train_test.drop(columns="month")
train_test = pd.concat([train_test, months], axis=1)

areas = pd.get_dummies(train_test["area"])
train_test = train_test.drop(columns="area")
train_test = pd.concat([train_test, areas], axis=1)

test_size = pd.read_csv("./data/test.csv").shape[0]
train = train_test[:-test_size]
test = train_test[-test_size:]

train = utils.get_target_values(train, target_vegetable)
test = utils.get_target_values(test, target_vegetable)

changed_col = [1, 0] + [i for i in np.arange(2, train.shape[1])]
train = train[:, changed_col]
test = test[:, changed_col]

test_amount = pd.read_csv("./data/test_amount.csv")
test_amount = test_amount[target_vegetable].values
test[:, 1] = test_amount

train_loader, train, test, ss = utils.preprocess_data_submit(train, test, T=param["T"], batch_size=param["batch_size"])

# Assign Free Params

In [46]:
future = test.shape[0]
num_epochs = param["num_epochs"]
learning_rate = param["learning_rate"]
weight_decay = param["weight_decay"]
eps = param["eps"]
hidden_size = param["hidden_size"]
dropout_ratio = param["dropout_ratio"]

# Training

In [47]:
model = utils.pipeline_rnn_submit(train_loader, train, test, future=future, num_epochs=num_epochs, lr=learning_rate,
                                  weight_decay=weight_decay, eps=eps, hidden_size=hidden_size, dropout_ratio=dropout_ratio)

# Predict

In [48]:
with torch.no_grad():
    model.eval()
    pred_y = model.predict(train, test, future)

pred_y = pred_y.cpu().detach().numpy()
pred_y = np.concatenate([pred_y.reshape(-1, 1), test[:, 1:]], axis=1)
pred_y[:, :7] = ss.inverse_transform(pred_y[:, :7])

print(f"std: {np.std(pred_y[:, 0].tolist())}")
pred_y[:, 0].tolist()

std: 527.1260690928439


[2916.013615022931,
 2869.310951050756,
 2706.9072743983975,
 2588.810049947525,
 2513.484024756102,
 2480.438655155315,
 2355.6128149142173,
 2292.7394116378528,
 2078.247063517784,
 1864.8188026786884,
 1739.6248749095419,
 1693.4806154809723,
 1613.3751863471277,
 1562.9617095863982,
 1510.1809745235892,
 1456.967337795485,
 1419.6595255957163,
 1421.7258570626186,
 1432.967977979954,
 1403.717935017923]